**Obbiettivo:**

- estrarre  le tabelle in zip  dalla struttura ad albero mantenendone la struttura ad albero nella cartella output salvandoli in **CSV**

- agregare per **symbol** e successivamente  per **giorno** calcolandone **IV** e **Expired Data**

In [ ]:
  # this should display information about available GPUs!nvidia-smi

In [ ]:
  # this should work without any errors

In [ ]:

import os
import zipfile
import pandas as pd



Mounted at /content/drive


# Extraction data

In [ ]:
"""import os
import zipfile


# qui c'è da metter mano e usare la programmazione multhi tread


base_path = #?
data_dir = os.path.join(base_path, 'fut_opt_taq_1min/2020')
dest_folder = os.path.join(base_path, 'output')
bad_list = []

# Cambia directory
os.chdir(data_dir)

# Itera attraverso tutte le directory e sottodirectory
for root, dirs, files in os.walk(data_dir):
    for file in files:
        if file.endswith('.zip'):
            file_path = os.path.join(root, file)
            try:
                with zipfile.ZipFile(file_path, 'r') as zip_ref:
                    # Crea la destinazione mantenendo la struttura delle cartelle
                    extract_path = os.path.join(dest_folder, os.path.relpath(root, data_dir))
                    os.makedirs(extract_path, exist_ok=True)
                    zip_ref.extractall(extract_path)
            except Exception as e:
                print(f"Error extracting {file_path}: {e}")
                bad_list.append(file_path)

# Stampa i file che non sono stati estratti correttamente
if bad_list:
    print("Files not extracted:")
    for bad_file in bad_list:
        print(bad_file)"""

# versione multi thread
import os
import zipfile
from concurrent.futures import ThreadPoolExecutor

base_path = #?
data_dir = os.path.join(base_path, 'fut_opt_taq_1min/2020')
dest_folder = os.path.join(base_path, 'output')
bad_list = []

def extract_zip(file_path):
    try:
        with zipfile.ZipFile(file_path, 'r') as zip_ref:
            # Crea la destinazione mantenendo la struttura delle cartelle nella cartella output
            extract_path = os.path.join(dest_folder, os.path.relpath(os.path.dirname(file_path), data_dir))
            os.makedirs(extract_path, exist_ok=True)
            zip_ref.extractall(extract_path)
        print(f"Extracted: {file_path}")
    except Exception as e:
        print(f"Error extracting {file_path}: {e}")
        bad_list.append(file_path)

def main():
    # change directory
    os.chdir(data_dir)

    zip_files = []
    # Iteriamo attraverso tutte le directory e sottodirectory
    for root, dirs, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.zip'):
                zip_files.append(os.path.join(root, file))

    # Usiamo il multithreading per estrarre i file ZIP
    with ThreadPoolExecutor() as executor:
        executor.map(extract_zip, zip_files)


    if bad_list:
        print("Files not extracted:")
        for bad_file in bad_list:
            print(bad_file)

if __name__ == "__main__":
    main()


Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/WB2/WB2H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/LO3/LO3H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/LO2/LO2H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/LO1/LO1H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/OG4/OG4H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/OG1/OG1H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/OG3/OG3H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/ZB2/ZB2H0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020/20200301/OZB/OZBM0.zip
Extracted: /content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_o

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import calendar

# Dizionari modificati ma mancano dei casi es OGJ,LO ecc


underlyings = {
    'LO': {'underlying': 'oil','expiry_desc':{
    'LO' : 'standard',
    'ML1': '1st mon', 'ML2': '2nd mon', 'ML3': '3rd mon', 'ML4': '4th mon', 'ML5': '5th mon',
    'WL1': '1st wed', 'WL2': '2nd wed', 'WL3': '3rd wed', 'WL4': '4th wed', 'WL5': '5th wed',
    'LO1': '1st fr', 'LO2': '2nd fr', 'LO3': '3rd fr', 'LO4': '4th fr', 'LO5': '5th fr'
    }},

    'OG': {'underlying': 'gold','expiry_desc':{
    'OG':  'standard',
    'G1M': '1st mon', 'G2M': '2nd mon', 'G3M': '3rd mon', 'G4M': '4th mon', 'G5M': '5th mon',
    'G1T': '1st tue', 'G2T': '2nd tue', 'G3T': '3rd tue', 'G4T': '4th tue', 'G5T': '5th tue',
    'G1W': '1st wed', 'G2W': '2nd wed', 'G3W': '3rd wed', 'G4W': '4th wed', 'G5W': '5th wed',
    'G1R': '1st thu', 'G2R': '2nd thu', 'G3R': '3rd thu', 'G4R': '4th thu', 'G5R': '5th thu',
    'OG1': '1st fri', 'OG2': '2nd fri', 'OG3': '3rd fri', 'OG4': '4th fri', 'OG5': '5th fri'
    }},

    'OZB': {'underlying': 'treasury','expiry_desc':{
    'OZB'  : 'standard',
    'VB1': '1st mon', 'VB2': '2nd mon', 'VB3': '3rd mon', 'VB4': '4th mon', 'VB5': '5th mon',
    'WB1': '1st wed', 'WB2': '2nd wed', 'WB3': '3rd wed', 'WB4': '4th wed', 'WB5': '5th wed',
    'ZB1': '1st fri', 'ZB2': '2nd fri', 'ZB3': '3rd fri', 'ZB4': '4th fri', 'ZB5': '5th fri'
    }}
}

In [ ]:

months = {'Jan':'F','Feb':'G','Mar':'H','Apr':'J','May':'K','Jun':'M','Jul':'N','Aug':'Q','Sep':'U','Oct':'V','Nov':'X','Dec':'Z'}
reverse_months = {v: k for k, v in months.items()}

# Funzioni per trovare le date specifiche
def nth_weekday(year, month, weekday, n):
    first_day = datetime(year, month, 1)
    # Handle the case where weekday is None
    if weekday is None:
        return None  # Or raise an error, or choose a default weekday
    first_weekday = first_day + timedelta(days=(weekday - first_day.weekday() + 7) % 7)
    return first_weekday + timedelta(weeks=(n-1))

# STABILIRE UNA POLICY DI COMPORTAMENTO QUANDO NON CI SONO LE CORRISPONDENZE
# complex O(n) --> complessità data da grandezza dizionario esterno passiamo da O(n^2) a O(n)
def find_underlying_and_key(lookup_key):
    for underlying, codes in underlyings.items():
        if lookup_key in codes['expiry_desc']: # più efficiente perchè facciamo solo 1 for e un ingresso diretto quindi non abbiamo un o(n^2) ma un o(n)
            return underlying, codes['underlying'], codes['expiry_desc'][lookup_key]
    return None, None, None


# complex O(1) --> complessità costante
def optCodeDecode(optCode):
  monthPlus = optCode[-2:] # new column
  monthCode = monthPlus[0]
  monthName = reverse_months[monthPlus[0]]
  optionClass = optCode[:-2]
#  print(f"underlying_code: {optionClass}")
  underlying, symbolName,expDate = find_underlying_and_key(optionClass)
#  print(f"underlying: {underlying}")
#  print(f"symbolName: {symbolName}")
#  print(f"expDate: {expDate}")
  # Decodifica del mese
  month_str = reverse_months[monthCode]
  month = list(months.keys()).index(month_str) + 1
  #print(f"month: {month} month_str: {month_str}")
#  print(f"desc: {expDate}")
  try:
    if expDate is None:
        raise ValueError("No matching key found in the dictionary")
    else:
        if 'mon' in expDate:
            weekday = 0
        elif 'tue' in expDate:
            weekday = 1
        elif 'wed' in expDate:
            weekday = 2
        elif 'thu' in expDate:
            weekday = 3
        elif 'fri' in expDate:
            weekday = 4
        else:
            weekday = 0  # Default to monday
        n = int(expDate.split()[0][0])
  except ValueError as e:
     print(f"Error: {e}")
     weekday = 0
     n = 0


#  print(f"weekday: {weekday} n: {n}")

  expiry_date = nth_weekday(year, month, weekday, n)
#  print(f"expiry_date: {expiry_date}")
  return monthPlus, monthCode,monthName,optionClass,underlying,symbolName,expDate,expiry_date


# complex O(1) --> complessità costante
def decode_ticker(ticker, year):
  optCode = ticker.split(' ')[0]
  sideStrike = ticker.split(' ')[1]
  side = sideStrike[0]
  strike = sideStrike[1:]
#  print(f"pt1 : {optCode}")
#  print(f"pt2 : {sideStrike}")
#  print(f"side : {side}")
#  print(f"strike : {strike}")
  monthPlus,monthCode,monthName,optionClass,symbol,symbolName,expDate,expiry_date = optCodeDecode(optCode)
  return symbol,symbolName,optionClass,strike,side,expiry_date

# Esempio di utilizzo funzione di decodifica
ticker = "OGJ0 C1350"
year = 2020
symbol,symbolName,optionClass,strike,side,expiry_date = decode_ticker(ticker, year)
print(f"symbol: {symbol}")
print(f"symbolName: {symbolName}")
print(f"optionClass: {optionClass}")
print(f"strike: {strike}")
print(f"side: {side}")
print(f"expiry_date: {expiry_date}")


Error: invalid literal for int() with base 10: 's'
symbol: OG
symbolName: gold
optionClass: OG
strike: 1350
side: C
expiry_date: 2020-03-30 00:00:00


In [ ]:
from tqdm import tqdm

def display_progress(iterable, total):
    with tqdm(total=total) as pbar:
        for item in iterable:
            yield item
            pbar.update(1)

# con l'uso dei thread si riesce a passare da 8 min a 5 min di esecuzione 37.5% di ottimizzazione!
def process_file(file_path):
    try:
        data = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    results = []
    for index, row in data.iterrows():
        symbol, symbolName, optionClass, strike, side, expiry_date = decode_ticker(row['Ticker'], 2020)
        results.append({
            'Symbol': symbol,
            'Underlying': symbolName,
            'Option Class': optionClass,
            'Strike': strike,
            'Side': side,
            'Expiry Date': expiry_date
        })
    results_df = pd.DataFrame(results)
    results_df['Ask'] = data['CloseAskPrice']
    results_df['Bid'] = data['CloseBidPrice']
    results_df['Min_spread'] = data['LowBidPrice'] - data['LowAskPrice']
    results_df['Max_spread'] = data['HighBidPrice'] - data['HighAskPrice']
    results_df['Volume/Trades'] = data['Volume'] / data['TotalTrades']
    # results_df['IV'] = results_df.apply(calculate_iv, axis=1)
    print(f"Processed {file_path}")
    return results_df

base_path = '/content/drive/MyDrive/Bogdan/Fondo/client_d474'
data_dir = os.path.join(base_path, 'fut_opt_taq_1min/2020')
dest_folder = os.path.join(base_path, 'output')
df_list = []

os.chdir(data_dir)

file_paths = []
for root, dirs, files in os.walk(dest_folder):
    for file in files:
        if file.endswith('.csv'):
            file_paths.append(os.path.join(root, file))

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    number_of_workers = os.cpu_count()  # Number of available CPU cores
    results = list(display_progress(executor.map(process_file, file_paths), total=len(file_paths)))


# Rimuovi i risultati None
df_list = [result for result in results if result is not None]

# Concatenare tutti i dataframe in uno solo
final_df = pd.concat(df_list, ignore_index=True)

print(final_df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020'

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
import calendar

months = {'Jan':'F','Feb':'G','Mar':'H','Apr':'J','May':'K','Jun':'M','Jul':'N','Aug':'Q','Sep':'U','Oct':'V','Nov':'X','Dec':'Z'}
reverse_months = {v: k for k, v in months.items()}

underlyings = {
    'LO': {'underlying': 'oil','expiry_desc':{
    'LO' : 'standard',
    'ML1': '1st mon', 'ML2': '2nd mon', 'ML3': '3rd mon', 'ML4': '4th mon', 'ML5': '5th mon',
    'WL1': '1st wed', 'WL2': '2nd wed', 'WL3': '3rd wed', 'WL4': '4th wed', 'WL5': '5th wed',
    'LO1': '1st fr', 'LO2': '2nd fr', 'LO3': '3rd fr', 'LO4': '4th fr', 'LO5': '5th fr'
    }},

    'OG': {'underlying': 'gold','expiry_desc':{
    'OG':  'standard',
    'G1M': '1st mon', 'G2M': '2nd mon', 'G3M': '3rd mon', 'G4M': '4th mon', 'G5M': '5th mon',
    'G1T': '1st tue', 'G2T': '2nd tue', 'G3T': '3rd tue', 'G4T': '4th tue', 'G5T': '5th tue',
    'G1W': '1st wed', 'G2W': '2nd wed', 'G3W': '3rd wed', 'G4W': '4th wed', 'G5W': '5th wed',
    'G1R': '1st thu', 'G2R': '2nd thu', 'G3R': '3rd thu', 'G4R': '4th thu', 'G5R': '5th thu',
    'OG1': '1st fri', 'OG2': '2nd fri', 'OG3': '3rd fri', 'OG4': '4th fri', 'OG5': '5th fri'
    }},

    'OZB': {'underlying': 'treasury','expiry_desc':{
    'OZB'  : 'standard',
    'VB1': '1st mon', 'VB2': '2nd mon', 'VB3': '3rd mon', 'VB4': '4th mon', 'VB5': '5th mon',
    'WB1': '1st wed', 'WB2': '2nd wed', 'WB3': '3rd wed', 'WB4': '4th wed', 'WB5': '5th wed',
    'ZB1': '1st fri', 'ZB2': '2nd fri', 'ZB3': '3rd fri', 'ZB4': '4th fri', 'ZB5': '5th fri'
    }}
}



class CSVProcessor2:
  def __init__(self, underlyings, months):
    self.underlyings = underlyings
    self.months = months

# Funzioni per trovare le date specifiche
  def nth_weekday(self,year, month, weekday, n):
    first_day = datetime(year, month, 1)
    # Handle the case where weekday is None
    if weekday is None:
        return None  # Or raise an error, or choose a default weekday
    first_weekday = first_day + timedelta(days=(weekday - first_day.weekday() + 7) % 7)
    return first_weekday + timedelta(weeks=(n-1))

# STABILIRE UNA POLICY DI COMPORTAMENTO QUANDO NON CI SONO LE CORRISPONDENZE
# complex O(n) --> complessità data da grandezza dizionario esterno passiamo da O(n^2) a O(n)
  def find_underlying_and_key(self,lookup_key):
    for underlying, codes in underlyings.items():
        if lookup_key in codes['expiry_desc']: # più efficiente perchè facciamo solo 1 for e un ingresso diretto quindi non abbiamo un o(n^2) ma un o(n)
            return underlying, codes['underlying'], codes['expiry_desc'][lookup_key]
    return None, None, None


# complex O(1) --> complessità costante
  def optCodeDecode(self,optCode):
    monthPlus = optCode[-2:] # new column
    monthCode = monthPlus[0]
    monthName = reverse_months[monthPlus[0]]
    optionClass = optCode[:-2]
#  print(f"underlying_code: {optionClass}")
    underlying, symbolName,expDate = self.find_underlying_and_key(optionClass)
#  print(f"underlying: {underlying}")
#  print(f"symbolName: {symbolName}")
#  print(f"expDate: {expDate}")
# Decodifica del mese
    month_str = reverse_months[monthCode]
    month = list(months.keys()).index(month_str) + 1
  #print(f"month: {month} month_str: {month_str}")
#  print(f"desc: {expDate}")
    try:
      if expDate is None:
          raise ValueError("No matching key found in the dictionary")
      else:
          if 'mon' in expDate:
            weekday = 0
          elif 'tue' in expDate:
            weekday = 1
          elif 'wed' in expDate:
            weekday = 2
          elif 'thu' in expDate:
            weekday = 3
          elif 'fri' in expDate:
            weekday = 4
          else:
            weekday = 0  # Default to monday
          n = int(expDate.split()[0][0])
    except ValueError as e:
     print(f"Error: {e}")
     weekday = 0
     n = 0
#  print(f"weekday: {weekday} n: {n}")

    expiry_date = self.nth_weekday(year, month, weekday, n)
#  print(f"expiry_date: {expiry_date}")
    return monthPlus, monthCode,monthName,optionClass,underlying,symbolName,expDate,expiry_date


# complex O(1) --> complessità costante
  def decode_ticker(self,ticker, year):
    optCode = ticker.split(' ')[0]
    sideStrike = ticker.split(' ')[1]
    side = sideStrike[0]
    strike = sideStrike[1:]
#  print(f"pt1 : {optCode}")
#  print(f"pt2 : {sideStrike}")
#  print(f"side : {side}")
#  print(f"strike : {strike}")
    monthPlus,monthCode,monthName,optionClass,symbol,symbolName,expDate,expiry_date = self.optCodeDecode(optCode)
    return symbol,symbolName,optionClass,strike,side,expiry_date

  def process_file(self,file_path):
    try:
        data = pd.read_csv(file_path)
    except Exception as e:
        print(f"Error reading {file_path}: {e}")
        return None
    results = []
    for index, row in data.iterrows():
        symbol,symbolName,optionClass,strike,side,expiry_date = decode_ticker(row['Ticker'], 2020)
        results.append({
            'Symbol': symbol,
            'Underlying': symbolName,
            'Option Class': optionClass,
            'Strike': strike,
            'Side': side,
            'Expiry Date': expiry_date
        })
    results_df = pd.DataFrame(results)
    results_df['Ask'] = data['CloseAskPrice']
    results_df['Bid'] = data['CloseBidPrice']
    results_df['Min_spread'] = data['LowBidPrice'] - data['LowAskPrice']
    results_df['Max_spread'] = data['HighBidPrice'] - data['HighAskPrice']
    results_df['Volume/Trades'] = data['Volume'] / data['TotalTrades']
    # results_df['IV'] = results_df.apply(calculate_iv, axis=1)
    print(f"Processed {file_path}")
    return results_df


# Esempio di utilizzo funzione di decodifica
ticker = "OGJ0 C1350"
year = 2020
fileProcessing = CSVProcessor2(underlyings, months)
symbol,symbolName,optionClass,strike,side,expiry_date = fileProcessing.decode_ticker(ticker, year)
print(f"symbol: {symbol}")
print(f"symbolName: {symbolName}")
print(f"optionClass: {optionClass}")
print(f"strike: {strike}")
print(f"side: {side}")
print(f"expiry_date: {expiry_date}")


Error: invalid literal for int() with base 10: 's'
symbol: OG
symbolName: gold
optionClass: OG
strike: 1350
side: C
expiry_date: 2020-03-30 00:00:00


In [ ]:
from tqdm import tqdm

def display_progress(iterable, total):
    with tqdm(total=total) as pbar:
        for item in iterable:
            yield item
            pbar.update(1)

In [ ]:


base_path = #?
data_dir = os.path.join(base_path, 'fut_opt_taq_1min/2020')
dest_folder = os.path.join(base_path, 'output')
df_list = []


os.chdir(data_dir)

file_paths = []
for root, dirs, files in os.walk(dest_folder):
    for file in files:
        if file.endswith('.csv'):
            file_paths.append(os.path.join(root, file))

with ThreadPoolExecutor(max_workers=os.cpu_count()) as executor:
    number_of_workers = os.cpu_count()  # Number of available CPU cores
    results = list(display_progress(executor.map(processFile.decode_ticker, file_paths), total=len(file_paths)))


# Rimuovi i risultati None
df_list = [result for result in results if result is not None]

# Concatenare tutti i dataframe in uno solo
final_df = pd.concat(df_list, ignore_index=True)

print(final_df.head())

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Bogdan/Fondo/client_d474/fut_opt_taq_1min/2020'

Store df for each simbol in a dict, you will be able to use data for each symbol using the name of option

NameError: name 'df' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
decoded_info.head()